In [1]:
# Import libs
import pandas as pd

In [2]:
# Load data
data = pd.read_csv('feedback_detail.csv')
data = pd.DataFrame(data)
data.head()

,GeneralFeedbackID,Title,Content,Upvote,Rating,CreatedDate,UserID,ProductID,FeedbackDetailID
0,19708302,Cực kì hài lòng,Đợt này mua tặng 2 cây trắng đen nhưng quên ch...,3,5,1704256818,5178168,184036446.0,1
1,19708307,Cực kì hài lòng,Đợt này mua tặng 2 cây trắng (giao từ Hanoi) đ...,2,5,1704256916,5178168,184059211.0,2
2,17356319,Cực kì hài lòng,"5* cho máy còn nguyên seal, hàng chính hãng VN...",184,5,1660043992,945720,123345348.0,3
3,19694107,Rất không hài lòng,"Mua xài mới có 1 ngày mà đã lỗi, có cuộc gọi t...",1,1,1703434010,15052027,197214029.0,4
4,18794713,Cực kì hài lòng,Chất lượng sản phẩm của Shop rất tốt đẹp full ...,20,5,1676287655,28830179,271966786.0,5


In [3]:
# Drop columns
data = data.drop(columns=['ProductID'])

In [4]:
data.columns

Index(['GeneralFeedbackID', 'Title', 'Content', 'Upvote', 'Rating',
       'CreatedDate', 'UserID', 'FeedbackDetailID'],
      dtype='object')

In [5]:
# Check the datatype
data.dtypes

GeneralFeedbackID     int64
Title                object
Content              object
Upvote                int64
Rating                int64
CreatedDate           int64
UserID                int64
FeedbackDetailID      int64
dtype: object

In [6]:
def categorize_feedback(feedback):
    if isinstance(feedback, str):
        product_keywords = ['sản phẩm', 'máy', 'hàng', 'chất lượng']
        service_keywords = ['giao hàng', 'đóng gói', 'shipper', 'shop']
        
        if any(keyword in feedback.lower() for keyword in product_keywords):
            return 'Product'
        elif any(keyword in feedback.lower() for keyword in service_keywords):
            return 'Service'
        else:
            return 'Unknown'
    else:
        return 'Unknown'

data['Category'] = data['Content'].apply(categorize_feedback)

In [7]:
category_counts = data['Category'].value_counts()
print("Feedback counts by category:")
print(category_counts)

Feedback counts by category:
Category
Unknown    6236
Product    2559
Service      79
Name: count, dtype: int64


In [9]:
from textblob import TextBlob

def analyze_sentiment(feedback):
    if isinstance(feedback, str):
        analysis = TextBlob(feedback)
        if analysis.sentiment.polarity > 0:
            return 'Positive'
        elif analysis.sentiment.polarity < 0:
            return 'Negative'
        else:
            return 'Neutral'
    else:
        return 'Unknown'

data['Sentiment'] = data['Content'].apply(analyze_sentiment)

In [10]:
print("Product feedback sentiment counts:")
print(data[data['Category'] == 'Product']['Sentiment'].value_counts())

print("\nService feedback sentiment counts:")
print(data[data['Category'] == 'Service']['Sentiment'].value_counts())

Product feedback sentiment counts:
Sentiment
Neutral     2123
Positive     347
Negative      89
Name: count, dtype: int64

Service feedback sentiment counts:
Sentiment
Neutral     69
Positive     8
Negative     2
Name: count, dtype: int64


In [19]:
import nltk
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer

def generate_summary(feedback):
    parser = PlaintextParser.from_string(feedback, Tokenizer("english"))
    summarizer = LsaSummarizer()
    summary = summarizer(parser.document, 3)  # Generate 3 sentence summary
    return ' '.join(str(sentence) for sentence in summary)

product_feedback = ' '.join(data[data['Category'] == 'Product']['Content'])
product_summary = generate_summary(product_feedback)

service_feedback = ' '.join(data[data['Category'] == 'Service']['Content'])
service_summary = generate_summary(service_feedback)

print("Product feedback summary:")
print(product_summary)

print("\nService feedback summary:")
print(service_summary)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kietd\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
c:\Users\kietd\AppData\Local\Programs\Python\Python312\Lib\site-packages\sumy\summarizers\lsa.py:76: UserWarning: Number of words (2369) is lower than number of sentences (2864). LSA algorithm may not work properly.
  warn(message % (words_count, sentences_count))


Product feedback summary:
sản phẩm giống quảng cáo, giao hàng nhanh đúng hẹn, máy hoạt động tốt, không gặp vấn đề gì, nhưng lúc bóc seal xong phát hiện có một vết xước nhẹ trên tay thỏ gần loa thoại chưa biết lý do, nên cho 3 sao thôi nhé. Hình chụp trăng ngoài cửa sổ sau 6 tháng dùng máy Tiki giao hàng nhanh, đóng gói cẩn thận, mình dùng được tuần rồi cũng chưa thấy vấn đề gì Tốt, sản phẩm chính hãng, có kích hoạt bảo ngay ngay mua thành công điện thoại chính hãng, và còn nguyên seal, mua vì cái form máy bo tròn mềm mại dễ bỏ túi chứ chức năng thì cũng k khác gì đời 12 và 13 Độ phân giải của điện thoại này rất tệ. -  Màn hinh iPhone 15 Pro Max có kích thước lớn, sử dụng công nghệ  Super Retina XDR cùng độ phân giải cao, mang tới chất lượng hình ảnh hiển thị sắc nét, chân thực và sống động.

Service feedback summary:
Mình mua không ngay sale, nhưng được voucher giảm 1,5tr thanh qua ví moca, và app shopback hoàn tiền thì giá cũng khá ổn so với ngoài store, và có được màu mình thích, chứ